# Pipeline

This notebook is from the official Quantopian Guide on Pipelines. Make sure to visit their documentation for many more great resources!

Many trading algorithms have the following structure:

1. For each asset in a known (large) set, compute N scalar values for the asset based on a trailing window of data.
2. Select a smaller tradeable set of assets based on the values computed in (1).
3. Calculate desired portfolio weights on the set of assets selected in (2).
4. Place orders to move the algorithm’s current portfolio allocations to the desired weights computed in (3).

There are several technical challenges with doing this robustly. These include:

* efficiently querying large sets of assets
* performing computations on large sets of assets
* handling adjustments (splits and dividends)
* asset delistings

Pipeline exists to solve these challenges by providing a uniform API for expressing computations on a diverse collection of datasets.

## Factors
A factor is a function from an asset and a moment in time to a numerical value.

A simple example of a factor is the most recent price of a security. Given a security and a specific point in time, the most recent price is a number. Another example is the 10-day average trading volume of a security. Factors are most commonly used to assign values to securities which can then be used in a number of ways. A factor can be used in each of the following procedures:
* computing target weights
* generating alpha signal
* constructing other, more complex factors
* constructing filters

## Filters
A filter is a function from an asset and a moment in time to a boolean.
An example of a filter is a function indicating whether a security's price is below $10. Given a security and a point in time, this evaluates to either True or False. Filters are most commonly used for describing sets of assets to include or exclude for some particular purpose.

## Classifiers
A classifier is a function from an asset and a moment in time to a categorical output.
More specifically, a classifier produces a string or an int that doesn't represent a numerical value (e.g. an integer label such as a sector code). Classifiers are most commonly used for grouping assets for complex transformations on Factor outputs. An example of a classifier is the exchange on which an asset is currently being traded.

In [1]:
from quantopian.pipeline import Pipeline

In [22]:
def make_pipeline():
    return Pipeline()

In [23]:
pipe = make_pipeline()

In [24]:
from quantopian.research import run_pipeline

In [25]:
result = run_pipeline(pipe,'2017-01-01','2017-01-01')

In [26]:
result.head(10)

Empty DataFrame
Columns: []
Index: [(2017-01-03 00:00:00+00:00, Equity(2 [ARNC])), (2017-01-03 00:00:00+00:00, Equity(21 [AAME])), (2017-01-03 00:00:00+00:00, Equity(24 [AAPL])), (2017-01-03 00:00:00+00:00, Equity(25 [ARNC_PR])), (2017-01-03 00:00:00+00:00, Equity(31 [ABAX])), (2017-01-03 00:00:00+00:00, Equity(39 [DDC])), (2017-01-03 00:00:00+00:00, Equity(41 [ARCB])), (2017-01-03 00:00:00+00:00, Equity(52 [ABM])), (2017-01-03 00:00:00+00:00, Equity(53 [ABMD])), (2017-01-03 00:00:00+00:00, Equity(62 [ABT]))]

In [27]:
result.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 8343 entries, (2017-01-03 00:00:00+00:00, Equity(2 [ARNC])) to (2017-01-03 00:00:00+00:00, Equity(50569 [OUSM]))
Empty DataFrame

# Data

In [36]:
from quantopian.pipeline.data.builtin import USEquityPricing

## Factors

Remember, Factors take in an asset and a timestamp and return some numerical value.

In [38]:
from quantopian.pipeline.factors import BollingerBands,SimpleMovingAverage,EWMA

In [40]:
SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30)

SimpleMovingAverage((USEquityPricing.close::float64,), window_length=30)

In [41]:
def make_pipeline():
    
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30)
    
    return Pipeline(columns={
        '30 Day Mean Close':mean_close_30
    })

In [46]:
results = run_pipeline(make_pipeline(),'2017-01-01','2017-01-01')

In [48]:
results.head(20)

30 Day Mean Close
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])              20.110500
                          Equity(21 [AAME])              3.899241
                          Equity(24 [AAPL])            113.368433
                          Equity(25 [ARNC_PR])          86.796111
                          Equity(31 [ABAX])             52.498394
                          Equity(39 [DDC])               9.523000
                          Equity(41 [ARCB])             29.969167
                          Equity(52 [ABM])              42.138239
                          Equity(53 [ABMD])            114.030167
                          Equity(62 [ABT])              38.664333
                          Equity(64 [ABX])              15.117736
                          Equity(66 [AB])               23.119167
                          Equity(67 [ADSK])             75.960667
                          Equity(69 [ACAT])             15.839500
                          Equity(70 [VBF])              18.208480
                          Equity(76 [TAP])              97.727924
                          Equity(84 [ACET])             20.722753
                          Equity(100 [IEP])             59.281167
                          Equity(106 [ACU])             22.628433
                          Equity(110 [ACXM])            26.800333

In [49]:
def make_pipeline():
    
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30)
    latest_close = USEquityPricing.close.latest
    
    return Pipeline(columns={
        '30 Day Mean Close':mean_close_30,
        'Latest Close':latest_close
    })

In [50]:
results = run_pipeline(make_pipeline(),'2017-01-01','2017-01-01')

In [52]:
results.head(10)

30 Day Mean Close  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])              20.110500   
                          Equity(21 [AAME])              3.899241   
                          Equity(24 [AAPL])            113.368433   
                          Equity(25 [ARNC_PR])          86.796111   
                          Equity(31 [ABAX])             52.498394   
                          Equity(39 [DDC])               9.523000   
                          Equity(41 [ARCB])             29.969167   
                          Equity(52 [ABM])              42.138239   
                          Equity(53 [ABMD])            114.030167   
                          Equity(62 [ABT])              38.664333   

                                                Latest Close  
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])             18.55  
                          Equity(21 [AAME])             4.10  
                          Equity(24 [AAPL])           115.84  
                          Equity(25 [ARNC_PR])           NaN  
                          Equity(31 [ABAX])            52.74  
                          Equity(39 [DDC])              9.69  
                          Equity(41 [ARCB])            27.75  
                          Equity(52 [ABM])             40.68  
                          Equity(53 [ABMD])           112.70  
                          Equity(62 [ABT])             38.42

## Combining Factors

In [54]:
def make_pipeline():
    
    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30)
    latest_close = USEquityPricing.close.latest
    
    percent_difference = (mean_close_10-mean_close_30) / mean_close_30
    
    return Pipeline(columns={
        'Percent Difference':percent_difference,
        '30 Day Mean Close':mean_close_30,
        'Latest Close':latest_close
    })

In [55]:
results = run_pipeline(make_pipeline(),'2017-01-01','2017-01-01')

In [56]:
results.head()

30 Day Mean Close  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])              20.110500   
                          Equity(21 [AAME])              3.899241   
                          Equity(24 [AAPL])            113.368433   
                          Equity(25 [ARNC_PR])          86.796111   
                          Equity(31 [ABAX])             52.498394   

                                                Latest Close  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])             18.55   
                          Equity(21 [AAME])             4.10   
                          Equity(24 [AAPL])           115.84   
                          Equity(25 [ARNC_PR])           NaN   
                          Equity(31 [ABAX])            52.74   

                                                Percent Difference  
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])               -0.022749  
                          Equity(21 [AAME])              -0.005499  
                          Equity(24 [AAPL])               0.028481  
                          Equity(25 [ARNC_PR])           -0.000474  
                          Equity(31 [ABAX])              -0.007665

# Filters and Screens

Filters take in an asset and a timestamp and return a boolean

In [57]:
last_close_price = USEquityPricing.close.latest
close_price_filter = last_close_price > 20

In [58]:
close_price_filter

NumExprFilter(expr='x_0 > (20.0)', bindings={'x_0': Latest((USEquityPricing.close::float64,), window_length=1)})

In [59]:
def make_pipeline():
    
    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30)
    latest_close = USEquityPricing.close.latest
    
    percent_difference = (mean_close_10-mean_close_30) / mean_close_30
    
    perc_diff_check = percent_difference > 0 
    
    return Pipeline(columns={
        'Percent Difference':percent_difference,
        '30 Day Mean Close':mean_close_30,
        'Latest Close':latest_close,
        'Positive Percent Diff': perc_diff_check
    })

In [60]:
results = run_pipeline(make_pipeline(),'2017-01-01','2017-01-01')
results.head()

30 Day Mean Close  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])              20.110500   
                          Equity(21 [AAME])              3.899241   
                          Equity(24 [AAPL])            113.368433   
                          Equity(25 [ARNC_PR])          86.796111   
                          Equity(31 [ABAX])             52.498394   

                                                Latest Close  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])             18.55   
                          Equity(21 [AAME])             4.10   
                          Equity(24 [AAPL])           115.84   
                          Equity(25 [ARNC_PR])           NaN   
                          Equity(31 [ABAX])            52.74   

                                                Percent Difference  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])               -0.022749   
                          Equity(21 [AAME])              -0.005499   
                          Equity(24 [AAPL])               0.028481   
                          Equity(25 [ARNC_PR])           -0.000474   
                          Equity(31 [ABAX])              -0.007665   

                                               Positive Percent Diff  
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])                     False  
                          Equity(21 [AAME])                    False  
                          Equity(24 [AAPL])                     True  
                          Equity(25 [ARNC_PR])                 False  
                          Equity(31 [ABAX])                    False

## Screens

In [61]:
def make_pipeline():
    
    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30)
    latest_close = USEquityPricing.close.latest
    
    percent_difference = (mean_close_10-mean_close_30) / mean_close_30
    
    perc_diff_check = percent_difference > 0 
    
    return Pipeline(columns={
                            'Percent Difference':percent_difference,
                            '30 Day Mean Close':mean_close_30,
                            'Latest Close':latest_close,
                            'Positive Percent Diff': perc_diff_check},
                    screen=perc_diff_check)

In [62]:
results = run_pipeline(make_pipeline(),'2017-01-01','2017-01-01')
results.head()

30 Day Mean Close  Latest Close  \
2017-01-03 00:00:00+00:00 Equity(24 [AAPL])         113.368433        115.84   
                          Equity(66 [AB])            23.119167         23.45   
                          Equity(69 [ACAT])          15.839500         15.02   
                          Equity(70 [VBF])           18.208480         18.49   
                          Equity(84 [ACET])          20.722753         21.97   

                                             Percent Difference  \
2017-01-03 00:00:00+00:00 Equity(24 [AAPL])            0.028481   
                          Equity(66 [AB])              0.004578   
                          Equity(69 [ACAT])            0.009375   
                          Equity(70 [VBF])             0.011814   
                          Equity(84 [ACET])            0.039630   

                                            Positive Percent Diff  
2017-01-03 00:00:00+00:00 Equity(24 [AAPL])                  True  
                          Equity(66 [AB])                    True  
                          Equity(69 [ACAT])                  True  
                          Equity(70 [VBF])                   True  
                          Equity(84 [ACET])                  True

### Reverse a screen

In [63]:
def make_pipeline():
    
    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30)
    latest_close = USEquityPricing.close.latest
    
    percent_difference = (mean_close_10-mean_close_30) / mean_close_30
    
    perc_diff_check = percent_difference > 0 
    
    return Pipeline(columns={
                            'Percent Difference':percent_difference,
                            '30 Day Mean Close':mean_close_30,
                            'Latest Close':latest_close,
                            'Positive Percent Diff': perc_diff_check},
                    screen=~perc_diff_check)

In [64]:
results = run_pipeline(make_pipeline(),'2017-01-01','2017-01-01')
results.head()

30 Day Mean Close  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])              20.110500   
                          Equity(21 [AAME])              3.899241   
                          Equity(25 [ARNC_PR])          86.796111   
                          Equity(31 [ABAX])             52.498394   
                          Equity(39 [DDC])               9.523000   

                                                Latest Close  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])             18.55   
                          Equity(21 [AAME])             4.10   
                          Equity(25 [ARNC_PR])           NaN   
                          Equity(31 [ABAX])            52.74   
                          Equity(39 [DDC])              9.69   

                                                Percent Difference  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])               -0.022749   
                          Equity(21 [AAME])              -0.005499   
                          Equity(25 [ARNC_PR])           -0.000474   
                          Equity(31 [ABAX])              -0.007665   
                          Equity(39 [DDC])               -0.015436   

                                               Positive Percent Diff  
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])                     False  
                          Equity(21 [AAME])                    False  
                          Equity(25 [ARNC_PR])                 False  
                          Equity(31 [ABAX])                    False  
                          Equity(39 [DDC])                     False

## Combine Filters

In [65]:
def make_pipeline():
    
    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30)
    latest_close = USEquityPricing.close.latest
    
    percent_difference = (mean_close_10-mean_close_30) / mean_close_30
    
    perc_diff_check = percent_difference > 0 
    small_price = latest_close < 5
    
    final_filter = perc_diff_check & small_price
    
    return Pipeline(columns={
                            'Percent Difference':percent_difference,
                            '30 Day Mean Close':mean_close_30,
                            'Latest Close':latest_close,
                            'Positive Percent Diff': perc_diff_check},
                    screen=final_filter)

In [66]:
results = run_pipeline(make_pipeline(),'2017-01-01','2017-01-01')
results.head()

30 Day Mean Close  Latest Close  \
2017-01-03 00:00:00+00:00 Equity(535 [ARTW])           3.097778          3.40   
                          Equity(677 [AXAS])           2.265333          2.56   
                          Equity(1144 [BTX])           3.531167          3.62   
                          Equity(1323 [CAW])           2.541333          2.60   
                          Equity(1546 [CIF])           2.500370          2.57   

                                              Percent Difference  \
2017-01-03 00:00:00+00:00 Equity(535 [ARTW])            0.013271   
                          Equity(677 [AXAS])            0.145527   
                          Equity(1144 [BTX])            0.065795   
                          Equity(1323 [CAW])            0.016002   
                          Equity(1546 [CIF])            0.015579   

                                             Positive Percent Diff  
2017-01-03 00:00:00+00:00 Equity(535 [ARTW])                  True  
                          Equity(677 [AXAS])                  True  
                          Equity(1144 [BTX])                  True  
                          Equity(1323 [CAW])                  True  
                          Equity(1546 [CIF])                  True

# Masking

Sometimes we want to ignore certain assets when computing pipeline expresssions. There are two common cases where ignoring assets is useful:
* We want to compute an expression that's computationally expensive, and we know we only care about results for certain assets.
* We want to compute an expression that performs comparisons between assets, but we only want those comparisons to be performed against a subset of all assets. 

In [83]:
def make_pipeline():
    
    # Create Filters for Masks First
    latest_close = USEquityPricing.close.latest
    small_price = latest_close < 5
    
    # Pass in the mask
    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10,mask=small_price)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30,mask=small_price)
    
    
    percent_difference = (mean_close_10-mean_close_30) / mean_close_30
    
    perc_diff_check = percent_difference > 0 
    
    
    final_filter = perc_diff_check
    
    return Pipeline(columns={
                            'Percent Difference':percent_difference,
                            '30 Day Mean Close':mean_close_30,
                            'Latest Close':latest_close,
                            'Positive Percent Diff': perc_diff_check},
                    screen=final_filter)

In [84]:
results = run_pipeline(make_pipeline(),'2017-01-01','2017-01-01')
results.head()

30 Day Mean Close  Latest Close  \
2017-01-03 00:00:00+00:00 Equity(535 [ARTW])           3.097778          3.40   
                          Equity(677 [AXAS])           2.265333          2.56   
                          Equity(1144 [BTX])           3.531167          3.62   
                          Equity(1323 [CAW])           2.541333          2.60   
                          Equity(1546 [CIF])           2.500370          2.57   

                                              Percent Difference  \
2017-01-03 00:00:00+00:00 Equity(535 [ARTW])            0.013271   
                          Equity(677 [AXAS])            0.145527   
                          Equity(1144 [BTX])            0.065795   
                          Equity(1323 [CAW])            0.016002   
                          Equity(1546 [CIF])            0.015579   

                                             Positive Percent Diff  
2017-01-03 00:00:00+00:00 Equity(535 [ARTW])                  True  
                          Equity(677 [AXAS])                  True  
                          Equity(1144 [BTX])                  True  
                          Equity(1323 [CAW])                  True  
                          Equity(1546 [CIF])                  True

In [85]:
len(results)

391

# Classifiers

A classifier is a function from an asset and a moment in time to a categorical output such as a string or integer label.

In [74]:
from quantopian.pipeline.data import morningstar
from quantopian.pipeline.classifiers.morningstar import Sector

In [75]:
morningstar_sector = Sector()

In [76]:
exchange = morningstar.share_class_reference.exchange_id.latest

In [77]:
exchange

Latest((share_class_reference.exchange_id::object,), window_length=1)

### Classifier Methods

* eq (equals)
* isnull
* startswith

In [79]:
nyse_filter = exchange.eq('NYS')

In [80]:
def make_pipeline():
    
    # Create Filters for Masks First
    latest_close = USEquityPricing.close.latest
    small_price = latest_close < 5
    
    # Classifier
    nyse_filter = exchange.eq('NYS')
    
    # Pass in the mask
    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10,mask=small_price)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30,mask=small_price)
    
    
    percent_difference = (mean_close_10-mean_close_30) / mean_close_30
    
    perc_diff_check = percent_difference > 0 
    
    
    final_filter = perc_diff_check & nyse_filter
    
    return Pipeline(columns={
                            'Percent Difference':percent_difference,
                            '30 Day Mean Close':mean_close_30,
                            'Latest Close':latest_close,
                            'Positive Percent Diff': perc_diff_check},
                    screen=final_filter)

In [81]:
results = run_pipeline(make_pipeline(),'2017-01-01','2017-01-01')
results.head()

30 Day Mean Close  Latest Close  \
2017-01-03 00:00:00+00:00 Equity(2586 [EQS])           1.960533         2.020   
                          Equity(3265 [GLF])           1.576367         1.725   
                          Equity(3645 [HOV])           2.406667         2.735   
                          Equity(4577 [LUB])           4.292333         4.270   
                          Equity(4971 [RT])            3.244000         3.240   

                                              Percent Difference  \
2017-01-03 00:00:00+00:00 Equity(2586 [EQS])            0.022120   
                          Equity(3265 [GLF])            0.162420   
                          Equity(3645 [HOV])            0.176939   
                          Equity(4577 [LUB])            0.004116   
                          Equity(4971 [RT])             0.009094   

                                             Positive Percent Diff  
2017-01-03 00:00:00+00:00 Equity(2586 [EQS])                  True  
                          Equity(3265 [GLF])                  True  
                          Equity(3645 [HOV])                  True  
                          Equity(4577 [LUB])                  True  
                          Equity(4971 [RT])                   True

In [82]:
len(results)

67

# Pipelines in Quantopian IDE

In [ ]:
from quantopian.pipeline import Pipeline
from quantopian.algorithm import attach_pipeline, pipeline_output

def initialize(context):
    my_pipe = make_pipeline()
    attach_pipeline(my_pipe, 'my_pipeline')

def make_pipeline():
    return Pipeline()

def before_trading_start(context, data):
    # Store our pipeline output DataFrame in context.
    context.output = pipeline_output('my_pipeline')